In [2]:
# Step 1: Install required libraries
!pip install fasttext transformers torch scikit-learn

import pandas as pd
from google.colab import files

# Step 2: Upload the CSV file
uploaded = files.upload()

# Step 3: Load the dataset into a DataFrame
df = pd.read_csv(next(iter(uploaded)))  # Assumes the first uploaded file is your dataset

# Map 'RequirementType' to 'labels' (Functional: 1, Non-Functional: 0)
label_mapping = {'F': 1, 'NF': 0}
df['labels'] = df['RequirementType'].map(label_mapping)

# Step 4: Clean text to remove newlines and extra spaces
def clean_text(text):
    # Remove any newline characters
    text = text.replace("\n", " ")
    # Optionally remove excess whitespace
    return ' '.join(text.split())

# Apply cleaning function to the 'content' column
df['cleaned_content'] = df['content'].apply(clean_text)

# Check if the 'labels' column and cleaned content are created correctly
print(df[['RequirementType', 'labels', 'cleaned_content']].head())

# Step 5: Download the FastText model from Hugging Face
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id="RafidMehda/fasttext_model", filename="fasttext_model.bin")

# Step 6: Load the fine-tuned FastText model
import fasttext
fasttext_model = fasttext.load_model(model_path)

# Step 7: Generate FastText embeddings for the cleaned dataset
def get_fasttext_embeddings(text):
    return fasttext_model.get_sentence_vector(text)

fasttext_embeddings = [get_fasttext_embeddings(doc) for doc in df['cleaned_content']]

# Step 8: Now merging this with DistilBERT
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def get_distilbert_embeddings(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get DistilBERT embeddings
    with torch.no_grad():
        outputs = distilbert_model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        pooled_embedding = torch.mean(last_hidden_state, dim=1)  # Average pooling of token embeddings
    return pooled_embedding.squeeze().numpy()

# Generate DistilBERT embeddings for the dataset
distilbert_embeddings = [get_distilbert_embeddings(doc) for doc in df['cleaned_content']]

# Step 9: Combine FastText and DistilBERT embeddings (concatenate)
import numpy as np
combined_embeddings = [np.concatenate((fasttext_emb, distilbert_emb)) for fasttext_emb, distilbert_emb in zip(fasttext_embeddings, distilbert_embeddings)]

# Step 10: Proceed with classification using the combined embeddings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_embeddings, df['labels'], test_size=0.2, random_state=42)

# Train a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Print detailed classification report
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=['Non-Functional', 'Functional']))



Saving final_corrected_fine_labeled_reviews.csv to final_corrected_fine_labeled_reviews (1).csv
  RequirementType  labels                                    cleaned_content
0               F       1                      I cannot open the app anymore
1              NF       0  I have been begging for a refund from this app...
2               F       1  Very costly for the premium version (approx In...
3              NF       0  Used to keep me organized, but all the 2020 UP...
4              NF       0                                Dan Birthday Oct 28


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Model Accuracy: 90.40%
Classification Report:

                precision    recall  f1-score   support

Non-Functional       0.91      0.92      0.91      1402
    Functional       0.89      0.89      0.89      1097

      accuracy                           0.90      2499
     macro avg       0.90      0.90      0.90      2499
  weighted avg       0.90      0.90      0.90      2499



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [4]:
# Save the fine-tuned model locally
import joblib
joblib.dump(classifier, './merge_fasttext_distilbert_model.pkl')

# Upload the model to Hugging Face
!pip install huggingface_hub
from huggingface_hub import notebook_login

# Login to your Hugging Face account
notebook_login()

from huggingface_hub import HfApi

# Upload the merged model to Hugging Face
api = HfApi()
api.upload_folder(
    folder_path='./',  # Path where the merged model is saved
    repo_id='RafidMehda/merge_fasttext_distilbert_model',  # Your Hugging Face repository
    repo_type='model'
)

merge_fasttext_distilbert_model.pkl:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

mnist_test.csv:   0%|          | 0.00/18.3M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

mnist_train_small.csv:   0%|          | 0.00/36.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RafidMehda/merge_fasttext_distilbert_model/commit/bbb50fe7608fd2b01174531ec15eec3e06234d6b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='bbb50fe7608fd2b01174531ec15eec3e06234d6b', pr_url=None, pr_revision=None, pr_num=None)